In [1]:
import pandas as pd
import json
import requests

In [2]:
# Get the 2017-2018 Capitals team roster from the NHL API.
res = requests.get('https://statsapi.web.nhl.com/api/v1/teams/15?expand=team.roster&season=20172018').text
json_roster = json.loads(res)

# We've got some ugly looking nested json to deal with here.
print(json.dumps(json_roster['teams'][0]['roster']['roster'][0], indent=4))

{
    "person": {
        "id": 8468498,
        "fullName": "Brooks Orpik",
        "link": "/api/v1/people/8468498"
    },
    "jerseyNumber": "44",
    "position": {
        "code": "D",
        "name": "Defenseman",
        "type": "Defenseman",
        "abbreviation": "D"
    }
}


In [3]:
# Convert the json object into a dataframe. Use json_normalize() for ugly nested json objs. 
df_roster = pd.json_normalize(json_roster['teams'][0]['roster']['roster'])

# Take note of the "id" key in the json above. That's going to help identify the players for the next part.
player_ids = df_roster['person.id']

df_roster.head(3)

,jerseyNumber,person.id,person.fullName,person.link,position.code,position.name,position.type,position.abbreviation
0,44,8468498,Brooks Orpik,/api/v1/people/8468498,D,Defenseman,Defenseman,D
1,8,8471214,Alex Ovechkin,/api/v1/people/8471214,L,Left Wing,Forward,LW
2,77,8471698,T.J. Oshie,/api/v1/people/8471698,R,Right Wing,Forward,RW


In [4]:
# There's a 'people' API endpoint we can feed 'person.id' into for more data.
players = []
for pid in player_ids:
    res = requests.get(f'https://statsapi.web.nhl.com/api/v1/people/{pid}').text
    json_person = json.loads(res)
    players.append(json_person['people'][0])
df_people = pd.DataFrame(players)
df_people.head(3)

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,birthCity,birthStateProvince,birthCountry,...,weight,active,rookie,shootsCatches,rosterStatus,primaryPosition,currentAge,alternateCaptain,captain,currentTeam
0,8468498,Brooks Orpik,/api/v1/people/8468498,Brooks,Orpik,44,1980-09-26,San Francisco,CA,USA,...,217,False,False,L,Y,"{'code': 'D', 'name': 'Defenseman', 'type': 'D...",NaN,NaN,NaN,NaN
1,8471214,Alex Ovechkin,/api/v1/people/8471214,Alex,Ovechkin,8,1985-09-17,Moscow,NaN,RUS,...,238,True,False,R,Y,"{'code': 'L', 'name': 'Left Wing', 'type': 'Fo...",35.0,False,True,"{'id': 15, 'name': 'Washington Capitals', 'lin..."
2,8471698,T.J. Oshie,/api/v1/people/8471698,T.J.,Oshie,77,1986-12-23,Everett,WA,USA,...,196,True,False,R,Y,"{'code': 'R', 'name': 'Right Wing', 'type': 'F...",34.0,False,False,"{'id': 15, 'name': 'Washington Capitals', 'lin..."


In [5]:
# Now to merge the two dataframes together keeping the rows correct with the player ids.
df_capitals = df_roster.merge(df_people, left_on='person.id', right_on='id', suffixes=('_p', None))
df_capitals.head(3)

,jerseyNumber,person.id,person.fullName,person.link,position.code,position.name,position.type,position.abbreviation,id,fullName,...,weight,active,rookie,shootsCatches,rosterStatus,primaryPosition,currentAge,alternateCaptain,captain,currentTeam
0,44,8468498,Brooks Orpik,/api/v1/people/8468498,D,Defenseman,Defenseman,D,8468498,Brooks Orpik,...,217,False,False,L,Y,"{'code': 'D', 'name': 'Defenseman', 'type': 'D...",NaN,NaN,NaN,NaN
1,8,8471214,Alex Ovechkin,/api/v1/people/8471214,L,Left Wing,Forward,LW,8471214,Alex Ovechkin,...,238,True,False,R,Y,"{'code': 'L', 'name': 'Left Wing', 'type': 'Fo...",35.0,False,True,"{'id': 15, 'name': 'Washington Capitals', 'lin..."
2,77,8471698,T.J. Oshie,/api/v1/people/8471698,R,Right Wing,Forward,RW,8471698,T.J. Oshie,...,196,True,False,R,Y,"{'code': 'R', 'name': 'Right Wing', 'type': 'F...",34.0,False,False,"{'id': 15, 'name': 'Washington Capitals', 'lin..."


In [6]:
# The NHL API is a treasure trove of data. Let's make this dataframe even bigger!
rows = []
for pid in player_ids:
    res = requests.get(f'https://statsapi.web.nhl.com/api/v1/people/{pid}/stats?stats=statsSingleSeason&season=20172018').text
    json_stats = json.loads(res)
    json_stats = json_stats['stats'][0]['splits'][0]['stat']

    # Need to add in the player id so we can merge it later.
    json_stats['stat.id'] = pid
    rows.append(json_stats)
    
# There are a lot of empty spots because some stats are only for goalies and vice versa.
df_stats = pd.DataFrame(rows)
df_stats.head(3)

,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,...,evenShots,powerPlayShots,savePercentage,goalAgainstAverage,gamesStarted,shotsAgainst,goalsAgainst,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage
0,1568:33,10.0,0.0,68.0,54.0,81,218.0,0.0,0.0,01:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1651:50,38.0,49.0,32.0,355.0,82,139.0,17.0,31.0,344:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1362:27,29.0,18.0,31.0,127.0,74,124.0,9.0,18.0,249:05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Let's merge this one in too.
df_capitals = df_capitals.merge(df_stats, left_on='person.id', right_on='stat.id', suffixes=('_s', None))
df_capitals.head(3)

,jerseyNumber,person.id,person.fullName,person.link,position.code,position.name,position.type,position.abbreviation,id,fullName,...,evenShots,powerPlayShots,savePercentage,goalAgainstAverage,gamesStarted,shotsAgainst,goalsAgainst,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage
0,44,8468498,Brooks Orpik,/api/v1/people/8468498,D,Defenseman,Defenseman,D,8468498,Brooks Orpik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,8471214,Alex Ovechkin,/api/v1/people/8471214,L,Left Wing,Forward,LW,8471214,Alex Ovechkin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,77,8471698,T.J. Oshie,/api/v1/people/8471698,R,Right Wing,Forward,RW,8471698,T.J. Oshie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Let's add even more data! Home-away game stat splits!
rows = []
for pid in player_ids:
    res = requests.get(f'https://statsapi.web.nhl.com/api/v1/people/{pid}/stats?stats=homeAndAway&season=20172018').text
    json_splits = json.loads(res)
    json_splits = json_splits['stats'][0]['splits']
    
    # check which stats are for home or away
    for gt in range(len(json_splits)):
        if(json_splits[gt]['isHome']):
            homeStats = json_splits[gt]['stat']
        else:
            awayStats = json_splits[gt]['stat']
    
    # now put the row together adding home. or away. to avoid duplicate column names
    totalStats = {'id': pid}
    for hkey in list(homeStats.keys()):
        totalStats['home.'+hkey] = homeStats[hkey]
    for akey in list(awayStats.keys()):
        totalStats['away.'+akey] = awayStats[akey]
    
    rows.append(totalStats)
    
# There are a lot of empty spots because some stats are only for goalies
df_splitStats = pd.DataFrame(rows)
df_splitStats.head(3)

,id,home.timeOnIce,home.assists,home.goals,home.pim,home.shots,home.games,home.hits,home.powerPlayGoals,home.powerPlayPoints,...,away.evenShots,away.powerPlayShots,away.savePercentage,away.goalAgainstAverage,away.gamesStarted,away.shotsAgainst,away.goalsAgainst,away.powerPlaySavePercentage,away.shortHandedSavePercentage,away.evenStrengthSavePercentage
0,8468498,787:51,2.0,0.0,31.0,28.0,41,118.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8471214,836:24,18.0,26.0,12.0,173.0,41,62.0,11.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8471698,669:38,15.0,6.0,16.0,65.0,36,69.0,4.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Let's add the split stats in too. And add a column for team name.
df_capitals = df_capitals.merge(df_splitStats, left_on='person.id', right_on='id', suffixes=('_split', None))
df_capitals['teamId'] = 15
df_capitals['teamLoc'] = 'Washington'
df_capitals['teamName'] = 'Capitals'
df_capitals

,jerseyNumber,person.id,person.fullName,person.link,position.code,position.name,position.type,position.abbreviation,id_split,fullName,...,away.goalAgainstAverage,away.gamesStarted,away.shotsAgainst,away.goalsAgainst,away.powerPlaySavePercentage,away.shortHandedSavePercentage,away.evenStrengthSavePercentage,teamId,teamLoc,teamName
0,44,8468498,Brooks Orpik,/api/v1/people/8468498,D,Defenseman,Defenseman,D,8468498,Brooks Orpik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
1,8,8471214,Alex Ovechkin,/api/v1/people/8471214,L,Left Wing,Forward,LW,8471214,Alex Ovechkin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
2,77,8471698,T.J. Oshie,/api/v1/people/8471698,R,Right Wing,Forward,RW,8471698,T.J. Oshie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
3,15,8471702,Matt Niskanen,/api/v1/people/8471702,D,Defenseman,Defenseman,D,8471702,Matt Niskanen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
4,44,8471710,Taylor Chorney,/api/v1/people/8471710,D,Defenseman,Defenseman,D,8471710,Taylor Chorney,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
5,19,8473563,Nicklas Backstrom,/api/v1/people/8473563,C,Center,Forward,C,8473563,Nicklas Backstrom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
6,16,8474061,Anthony Peluso,/api/v1/people/8474061,R,Right Wing,Forward,RW,8474061,Anthony Peluso,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
7,20,8474189,Lars Eller,/api/v1/people/8474189,C,Center,Forward,C,8474189,Lars Eller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
8,83,8474291,Jay Beagle,/api/v1/people/8474291,C,Center,Forward,C,8474291,Jay Beagle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals
9,74,8474590,John Carlson,/api/v1/people/8474590,D,Defenseman,Defenseman,D,8474590,John Carlson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Washington,Capitals


In [11]:
df_capitals.to_csv('dirty-caps-data.csv', index=False)